Este projeto visa a aplicação de técnicas de tratamento de dados utilizando os dados do portal CEAPS, que contém informações sobre os gastos dos senadores brasileiros. 

As tarefas executadas incluem a identificação de inconsistências nos dados, como valores nulos e duplicados, a correção de campos monetários e a conversão de datas em formatos adequados. 

O objetivo é preparar os dados para análises mais aprofundadas, garantindo a qualidade e a integridade das informações.

## Tecnologias Utilizadas
Python 3.12

Pandas

Matplotlib

Statsmodels

Jupyter Notebook

Docker

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import requests
import time
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

1. Construção do dataframe

In [ ]:
arquivos = ['../docs/despesa_ceaps_2019.csv', '../docs/despesa_ceaps_2020.csv', '../docs/despesa_ceaps_2021.csv', '../docs/despesa_ceaps_2022.csv']
dataframes = []
for arquivo in arquivos:
    df = pd.read_csv(arquivo, header=1)  
    dataframes.append(df)
dataset = pd.concat(dataframes, ignore_index=True)

2. Visualização dos dados

In [ ]:
# Preview do dataframe (5 primeiras linhas)
dataset.head()

In [ ]:
# Tamanho do dataframe
dataset.shape

In [ ]:
# Nome das variáveis
print(dataset.columns)

In [ ]:
# Tipos das variáveis
print(dataset.dtypes)

In [ ]:
#Conferência se estão os dados dos 4 anos importados no csv
print(dataset['ANO'].unique())

In [ ]:
#Conferência se estão os dados dos 4 anos importados no csv
print(dataset['MES'].unique())

3. Tratamento dos dados nulos

In [ ]:
#Contagem dos nulos
dataset.isnull().sum()

Agora ficou claro que existem dados faltantes nas variáveis DOCUMENTO, DETALHAMENTO e COD_DOCUMENTO. A seguir sera feita uma análise de cada variável para decidir como proceder nos casos dos dados faltantes.

In [ ]:
# Exibir os valores únicos na coluna DOCUMENTO
print("Valores únicos na coluna DOCUMENTO:")
print(dataset['DOCUMENTO'].unique())

# Contar a frequência de cada valor na coluna DOCUMENTO
print("Frequência de cada valor na coluna DOCUMENTO:")
print(dataset['DOCUMENTO'].value_counts())

Os dados contidos nesta variável trazem informações vagas sobre qual seria o documento. Porem o mesmo não segue um padrão e tem vários dados faltantes. Sabendo disso, podemos afirmar que esta variável não é relevante para este projeto.

In [ ]:
dataset[dataset['DETALHAMENTO'].isna()].head(5)

A variável DETALHAMENTO segue o mesmo mencionado para DOCUMENTO. Pelo mesmo motivo, também sera removida.

In [ ]:
dataset = dataset.drop(columns=['DOCUMENTO', 'DETALHAMENTO'])

#Checagem após a exclusão das variáveis
print(dataset.columns)

In [ ]:
#Distribuição dos dados na variável Senador
dataset['SENADOR'].value_counts()

Temos 159 senadores, sendo que o Carlos Viana é o que aparece com mais registros.

In [ ]:
dataset['COD_DOCUMENTO'].value_counts()

In [ ]:
#Como temos apenas 1 registro nulo na variável COD_DOCUMENTO, esta sera removida
dataset = dataset.dropna(subset=['COD_DOCUMENTO'])

In [ ]:
# Esta variável é uma informação do tipo texto e não numerica como consta. Para esta não comprometar futuras visualizações dos dados, o correto é converter a mesma para string
dataset['COD_DOCUMENTO'] = dataset['COD_DOCUMENTO'].astype(str)


In [ ]:
#Checagem dos dados faltantes:
dataset.isnull().sum()

4. Tratamento da coluna ANO e DATA

Há um erro de registro nas datas. Antes de converter essa string em date, é preciso corrigir esse erro.

In [ ]:
# Contagem de quantas vezes o ano esta escrito de forma errada:
mascara = dataset['DATA'].str.contains(r'/02\d{2}', na=False)

quantidade = mascara.sum()

print("Quantidade de vezes que aparece '/02' quando deveria ser '/20' no lugar do ano:", quantidade)


In [ ]:
# Substituição de todos os registros errado nos anos:
dataset['DATA'] = dataset['DATA'].str.replace(r'/02(\d{2})$', r'/20\1', regex=True)

In [ ]:
dataset['DATA'] = pd.to_datetime(dataset['DATA'], format='%d/%m/%Y')

Como este projeto trata apenas de 2019 a 2022, qualquer informação fora disso sera removida.

In [ ]:
#Verificar se possem datas diferente de 2019 até 2022 na variável DATA:
# Definir o intervalo de datas
inicio_intervalo = pd.to_datetime('2019-01-01')
fim_intervalo = pd.to_datetime('2022-12-31')

# Filtrar o DataFrame para remover datas fora do intervalo usando inplace=True
dataset.drop(dataset[(dataset['DATA'] < inicio_intervalo) | (dataset['DATA'] > fim_intervalo)].index, inplace=True)
print("DataFrame após remover registros fora do intervalo 2019-2022:")
print(dataset['DATA'])

In [ ]:
# Identificar datas fora do intervalo
datas_fora_intervalo = dataset[(dataset['DATA'] < inicio_intervalo) | (dataset['DATA'] > fim_intervalo)]

# Imprimir apenas as informações da coluna DATA que estão fora do intervalo 2019-2022
print("Datas fora do intervalo 2019-2022:")
print(datas_fora_intervalo['DATA'])


Como ja temos as informações de datas na variável DATA, não precisamos manter as variáveis ANO e MES, pois são redundantes.

In [ ]:
dataset.drop(columns=['ANO', 'MES'], inplace=True)

In [ ]:
# Checagem das variáveis
print(dataset.dtypes)

In [ ]:
dataset.head(20)

5. Inspeção da variável VALOR_REEMBOLSADO

In [ ]:
#Substituição da vírgula para ponto, e em seguida a conversão de string para float
dataset['VALOR_REEMBOLSADO'] = dataset['VALOR_REEMBOLSADO'].str.replace(',', '.')
dataset['VALOR_REEMBOLSADO'] = dataset['VALOR_REEMBOLSADO'].astype(float)


In [ ]:
# Garantir que VALOR_REEMBOLSADO está no formato numérico
#dataset['VALOR_REEMBOLSADO'] = pd.to_numeric(dataset['VALOR_REEMBOLSADO'], errors='coerce')

# Calcular as estatísticas
valor_max = dataset['VALOR_REEMBOLSADO'].max()
valor_min = dataset['VALOR_REEMBOLSADO'].min()
valor_media = dataset['VALOR_REEMBOLSADO'].mean()
valor_mediana = dataset['VALOR_REEMBOLSADO'].median()

# Exibir os resultados
print(f"Valor Máximo: R$ {valor_max:.2f}")
print(f"Valor Mínimo: R$ {valor_min:.2f}")
print(f"Valor Médio: R$ {valor_media:.2f}")
print(f"Mediana: R$ {valor_mediana:.2f}")


In [ ]:
# Calcular Q1, Q3 e IQR
Q1 = dataset['VALOR_REEMBOLSADO'].quantile(0.25)
Q3 = dataset['VALOR_REEMBOLSADO'].quantile(0.75)
IQR = Q3 - Q1

# Definir os limites inferior e superior
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar os dados dentro do intervalo
dataset_sem_outliers = dataset[(dataset['VALOR_REEMBOLSADO'] >= limite_inferior) & 
                               (dataset['VALOR_REEMBOLSADO'] <= limite_superior)]

# Exibir as novas estatísticas
print(f"Dados sem outliers: {len(dataset_sem_outliers)} linhas")
print(f"Valor Máximo (Ajustado): R$ {dataset_sem_outliers['VALOR_REEMBOLSADO'].max():.2f}")
print(f"Valor Mínimo (Ajustado): R$ {dataset_sem_outliers['VALOR_REEMBOLSADO'].min():.2f}")


In [ ]:
# Selecionar apenas os outliers
outliers = dataset[(dataset['VALOR_REEMBOLSADO'] < limite_inferior) | 
                   (dataset['VALOR_REEMBOLSADO'] > limite_superior)]

# Exibir os outliers
print(f"Número de outliers: {len(outliers)}")
print(outliers[['VALOR_REEMBOLSADO']].describe())

In [ ]:
# Exibir os outliers
print(f"Número de outliers: {len(outliers)}")
print(outliers[['VALOR_REEMBOLSADO']].describe())

# Plotar os valores de reembolso e destacar os outliers
plt.figure(figsize=(10, 6))
plt.plot(dataset['DATA'], dataset['VALOR_REEMBOLSADO'], label='Valor Reembolsado', color='blue', marker='o')
plt.scatter(outliers['DATA'], outliers['VALOR_REEMBOLSADO'], color='red', label='Outliers', zorder=5)
plt.axhline(limite_inferior, color='green', linestyle='--', label='Limite Inferior')
plt.axhline(limite_superior, color='orange', linestyle='--', label='Limite Superior')
plt.xlabel('Data')
plt.ylabel('Valor Reembolsado')
plt.title('Variação do Valor Reembolsado com Outliers Destacados')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Selecionar apenas os outliers
outliers = dataset[(dataset['VALOR_REEMBOLSADO'] < limite_inferior) | (dataset['VALOR_REEMBOLSADO'] > limite_superior)]

# Plotar os valores de reembolso e destacar os outliers
plt.figure(figsize=(10, 6))
plt.plot(dataset.index, dataset['VALOR_REEMBOLSADO'], label='Valor Reembolsado', color='blue', marker='o')
plt.scatter(outliers.index, outliers['VALOR_REEMBOLSADO'], color='red', label='Outliers', zorder=5)
plt.axhline(limite_inferior, color='green', linestyle='--', label='Limite Inferior')
plt.axhline(limite_superior, color='orange', linestyle='--', label='Limite Superior')
plt.xlabel('Índice')
plt.ylabel('Valor Reembolsado')
plt.title('Variação do Valor Reembolsado com Outliers Destacados')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Criar bins (intervalos)
bins = [0, 100, 500, 1000, 5000, 10000, 50000, dataset['VALOR_REEMBOLSADO'].max()]
labels = ['0-100', '101-500', '501-1000', '1001-5000', '5001-10000', '10001-50000', '50001+']
dataset['VALOR_REEMBOLSADO_BINS'] = pd.cut(dataset['VALOR_REEMBOLSADO'], bins=bins, labels=labels)

# Contar frequência em cada bin
print(dataset['VALOR_REEMBOLSADO_BINS'].value_counts())

# Visualizar
dataset['VALOR_REEMBOLSADO_BINS'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribuição por Intervalos de Valor')
plt.xlabel('Intervalos de Valor Reembolsado')
plt.ylabel('Frequência')
plt.show()


6. Visualização dos dados

Já que temos os dados prontos para uma breve analise, veja alguns dados e gráficos a seguir.

In [ ]:
# Número de dados na variável 'TIPO_DESPESA'
numero_tipos_despesa = dataset['TIPO_DESPESA'].nunique()
print(f"Número de tipos únicos de despesa: {numero_tipos_despesa}")


In [ ]:
# Os 10 tipos mais frequentes e quantas vezes eles aparecem no dataset.
contagem_despesas = dataset['TIPO_DESPESA'].value_counts()

# Visualização em forma tabular
tabela_despesas = pd.DataFrame(contagem_despesas)
tabela_despesas.reset_index(inplace=True)
tabela_despesas.columns = ['Tipo de Despesa', 'Contagem']
print(tabela_despesas)



Agora uma breve analise em relação ao reembolso dos senadores.

In [ ]:
# Agrupar os dados por CNPJ_CPF e somar os VALOR_REEMBOLSADO
cnpj_total = dataset.groupby('CNPJ_CPF')['VALOR_REEMBOLSADO'].sum().reset_index()

# Ordenar os CNPJs pelo valor reembolsado em ordem decrescente e pegar os top 10
top10_cnpjs = cnpj_total.sort_values(by='VALOR_REEMBOLSADO', ascending=False).head(10)

# Exibir os resultados
print(top10_cnpjs)


In [ ]:
def limpar_cnpj(cnpj):
    # Remover quaisquer caracteres que não sejam dígitos
    return ''.join(filter(str.isdigit, cnpj))

def obter_nome_empresa(cnpj):
    cnpj = limpar_cnpj(cnpj)  # Limpa o CNPJ antes de fazer a requisição
    url = f"https://brasilapi.com.br/api/cnpj/v1/{cnpj}"
    tentativas = 3  # Número de tentativas de requisição
    for tentativa in range(tentativas):
        try:
            response = requests.get(url, timeout=10)  # Define um timeout de 10 segundos
            response.raise_for_status()  # Verifica se a requisição foi bem-sucedida
            data = response.json()
            return data.get('razao_social')
        except requests.HTTPError as e:
            print(f"Erro HTTP ao acessar a API para o CNPJ {cnpj}: {e}")
            if response.status_code in [500, 502, 503, 504]:
                time.sleep(5)  # Espera 5 segundos antes de tentar novamente se for erro do servidor
            else:
                break  # Para outros erros HTTP, interrompe as tentativas
        except requests.RequestException as e:
            print(f"Erro de rede ao acessar a API para o CNPJ {cnpj}: {e}")
            time.sleep(5)  # Espera 5 segundos antes de tentar novamente
        except Exception as e:
            print(f"Erro desconhecido: {e}")
            break
    return None

# Aplicação da função com tratamento de exceções
top10_cnpjs['NOME_EMPRESA'] = top10_cnpjs['CNPJ_CPF'].apply(obter_nome_empresa)

# Verificar o DataFrame
print(top10_cnpjs[['CNPJ_CPF', 'NOME_EMPRESA']])

# Criar o gráfico de barras com Plotly Express
fig = px.bar(top10_cnpjs, x="NOME_EMPRESA", y="VALOR_REEMBOLSADO",
             title="Top 10 Empresas por Valor Reembolsado",
             labels={"VALOR_REEMBOLSADO": "Valor Reembolsado Total (R$)", "NOME_EMPRESA": "Empresa"},
             text='VALOR_REEMBOLSADO')

# Ajustar a apresentação do gráfico
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(xaxis={'categoryorder':'total descending'},
                  yaxis_title="Valor Reembolsado Total (R$)",
                  xaxis_title="Empresa",
                  xaxis_tickangle=-45)

# Exibir o gráfico
fig.show()


In [ ]:
# Filtrar o dataset para incluir apenas registros do CNPJ específico
cnpj_especifico = '02.354.051/0001-47'
dados_filtrados = dataset[dataset['CNPJ_CPF'] == cnpj_especifico]

# Agrupar os dados por SENADOR e somar os VALOR_REEMBOLSADO
senadores_reembolsos = dados_filtrados.groupby('SENADOR')['VALOR_REEMBOLSADO'].sum().reset_index()

# Ordenar os senadores pelo valor reembolsado em ordem decrescente
senadores_reembolsos = senadores_reembolsos.sort_values(by='VALOR_REEMBOLSADO', ascending=False)

# Exibir os resultados
print(f"O(s) senador(es) com o maior(es) reembolso(s) para a empresa 14.294.443/0001-47 (A CAMACHO TORRES) foi o: {senadores_reembolsos}")


In [ ]:
# Filtrar o dataset para incluir apenas registros do CNPJ específico
cnpj_especifico = '33.055.400/0001-53'
dados_filtrados = dataset[dataset['CNPJ_CPF'] == cnpj_especifico]

# Agrupar os dados por SENADOR e somar os VALOR_REEMBOLSADO
senadores_reembolsos = dados_filtrados.groupby('SENADOR')['VALOR_REEMBOLSADO'].sum().reset_index()

# Ordenar os senadores pelo valor reembolsado em ordem decrescente
senadores_reembolsos = senadores_reembolsos.sort_values(by='VALOR_REEMBOLSADO', ascending=False)

# Exibir os resultados
print(f"O(s) senador(es) com o maior(es) reembolso(s) para a empresa 33.055.400/0001-53 (L COELHO SERRA) foi o: {senadores_reembolsos}")


In [ ]:
# Filtrar o dataset para incluir apenas registros do CNPJ específico
cnpj_especifico = '31.837.019/0001-10'
dados_filtrados = dataset[dataset['CNPJ_CPF'] == cnpj_especifico]

# Agrupar os dados por SENADOR e somar os VALOR_REEMBOLSADO
senadores_reembolsos = dados_filtrados.groupby('SENADOR')['VALOR_REEMBOLSADO'].sum().reset_index()

# Ordenar os senadores pelo valor reembolsado em ordem decrescente
senadores_reembolsos = senadores_reembolsos.sort_values(by='VALOR_REEMBOLSADO', ascending=False)

# Exibir os resultados
print(f"O(s) senador(es) com o maior(es) reembolso(s) para a empresa 31.837.019/0001-10 (BORA COMUNICACAO E MARKETING DIGITAL LTDA) foi o: {senadores_reembolsos}")


In [ ]:
#Função para abreviar os valores que será usada em alguns gráficos a seguir.
def abreviar_valor(num):
    """Abrevia valores numéricos usando K para milhares, M para milhões, etc."""
    for unidade in ['', 'K', 'M', 'B', 'T']:
        if abs(num) < 1000:
            return f"{num:.1f}{unidade}"
        num /= 1000.0
    return f"{num:.1f}T"  # Caso extremamente grande

In [ ]:
# Calcular a soma total dos valores reembolsados
total_reembolsos = dataset['VALOR_REEMBOLSADO'].sum()

# Exibir o valor total e o valor abreviado
print(f"O valor total dos reembolsos é: R$ {total_reembolsos:.2f}")
print(f"O valor total dos reembolsos abreviado é: R$ {abreviar_valor(total_reembolsos)}")


In [ ]:
# Calcular a soma total dos valores reembolsados
total_reembolsos = dataset['VALOR_REEMBOLSADO'].sum()

# Criar o gráfico de barra
fig = go.Figure(go.Bar(
    x=['Total Reembolsos'],
    y=[total_reembolsos],
    text=[f"R$ {abreviar_valor(total_reembolsos)}"],
    textposition='auto',
))

# Ajustar a apresentação do gráfico
fig.update_layout(
    title="Valor Total dos Reembolsos",
    yaxis_title="Valor Reembolsado Total (R$)",
    xaxis_title="",
    showlegend=False
)

# Exibir o gráfico
fig.show()


In [ ]:
# Extrair o ano da coluna 'DATA'
dataset['ANO'] = dataset['DATA'].dt.year

# Agrupar os dados por ano e calcular a soma dos valores reembolsados
soma_ano = dataset.groupby('ANO')['VALOR_REEMBOLSADO'].sum().reset_index()

# Função para abreviar os valores
def abreviar_valor(val):
    if val >= 1e6:
        return f'{val/1e6:.1f}M'
    elif val >= 1e3:
        return f'{val/1e3:.1f}K'
    return str(val)

# Criar o gráfico
plt.figure(figsize=(10, 6))
bars = plt.bar(soma_ano['ANO'].astype(str), soma_ano['VALOR_REEMBOLSADO'], color='skyblue')

# Adicionar rótulos com valores abreviados acima de cada barra
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, abreviar_valor(yval), va='bottom', ha='center')

plt.xlabel('Ano')
plt.ylabel('Valor Reembolsado')
plt.title('Soma dos Reembolsos por Ano')
plt.xticks(soma_ano['ANO'].astype(str))  # Configura explicitamente os rótulos do eixo X

plt.tight_layout()
plt.show()


A seguir vamos ver a relação dos senadores com esses reembolsos.

In [ ]:
# Agrupar e somar por ano e senador
df_agg = dataset.groupby(['SENADOR', 'ANO'])['VALOR_REEMBOLSADO'].sum().reset_index()

# Selecionar os últimos 4 anos
anos_disponiveis = sorted(df_agg['ANO'].unique())
ultimos_4_anos = anos_disponiveis[-4:]

# Filtrar apenas os últimos 4 anos
df_ultimos_4 = df_agg[df_agg['ANO'].isin(ultimos_4_anos)]

# Top 10 senadores por cada ano
top_10_por_ano = df_ultimos_4.groupby('ANO', group_keys=False) \
                             .apply(lambda x: x.nlargest(10, 'VALOR_REEMBOLSADO')) \
                             .reset_index(drop=True)

# Arredondar VALOR_REEMBOLSADO para 2 casas decimais
top_10_por_ano['VALOR_REEMBOLSADO'] = top_10_por_ano['VALOR_REEMBOLSADO'].round(2)

# Criar o gráfico com Plotly, um ano abaixo do outro
# sharex=False garante que cada subplot terá seu próprio eixo X
fig = px.bar(
    top_10_por_ano,
    x='SENADOR',
    y='VALOR_REEMBOLSADO',
    facet_row='ANO',
    title='Top 10 Senadores por Ano (Últimos 4 Anos)',
    height=1200,
    facet_row_spacing=0.05,   # Ajuste o espaçamento vertical entre os subplots se necessário
)

# Manter os rótulos alinhados e legíveis
fig.update_xaxes(tickangle=90, automargin=True)
fig.update_yaxes(tickformat='.2f', automargin=True)

# Desabilitar eixo X compartilhado para repetir os nomes de senadores em cada subplot
fig.update_layout(
    title_x=0.5,
    showlegend=False,  # Caso não queira legenda repetida
)

# Mostrar o gráfico
fig.show()


Neste grafico vemos os vereadores que mais tiverem reembolsos entre os anos 2019 e 2022. Nesta disposição é possível identificar também os senadores que estiveram no top 10 nos 4 anos seguidos, sendo eles: MECIAS DE JESUS, ROGÉRIO CARVALHO e TELMÁRIO MOTA.

Vamos analisar esses três senadores em separado, para ver o montante dos reembolsos deles em relação ao total entre 2019 e 2022.

In [ ]:
# Nomes dos senadores de interesse
senadores = ['MECIAS DE JESUS', 'ROGÉRIO CARVALHO', 'TELMÁRIO MOTA']

# Filtrar os dados para esses senadores e somar os valores reembolsados
soma_senadores = dataset[dataset['SENADOR'].isin(senadores)]['VALOR_REEMBOLSADO'].sum()

# Converter VALOR_REEMBOLSADO para float, se ainda não está convertido
dataset['VALOR_REEMBOLSADO'] = dataset['VALOR_REEMBOLSADO'].astype(float)

# Calcular o total de VALOR_REEMBOLSADO no dataset
total_reembolso = dataset['VALOR_REEMBOLSADO'].sum()

# Calcular a porcentagem
porcentagem = (soma_senadores / total_reembolso) * 100

# Criar um gráfico de barras para mostrar a comparação
fig, ax = plt.subplots(figsize=(8, 6))

# Dados para o gráfico
categorias = ['Selecionados', 'Total']
valores = [soma_senadores, total_reembolso]

ax.bar(categorias, valores, color=['blue', 'gray'])
ax.set_ylabel('Valor Reembolsado')
ax.set_title('Comparação de Reembolso: Senadores Selecionados vs Total')

# Mostrar a porcentagem sobre a barra correspondente
ax.text(0, soma_senadores, f'{porcentagem:.2f}%', ha='center', va='bottom', color='black')

plt.tight_layout()
plt.show()


Agora uma pequena análise para saber quem foram os top 10 que mais receberam reembolsos nos 4 anos do dataset e quanto eles receberam.

In [ ]:
# Somar os reembolsos dos últimos 4 anos por senador utilizando a função para abreviar os valores
df_4_anos = df_ultimos_4.groupby('SENADOR')['VALOR_REEMBOLSADO'].sum().reset_index()
df_4_anos = df_4_anos.nlargest(10, 'VALOR_REEMBOLSADO')

plt.figure(figsize=(8,6))
ax = sns.barplot(
    data=df_4_anos,
    x='SENADOR',
    y='VALOR_REEMBOLSADO'
)

# Adicionar os valores abreviados dentro de cada barra
for p in ax.patches:
    height = p.get_height()
    valor_abreviado = abreviar_valor(height)
    ax.annotate(valor_abreviado,
                xy=(p.get_x() + p.get_width() / 2, height),
                xytext=(0, 5),
                textcoords='offset points',
                ha='center', va='bottom',
                color='black', fontsize=9)

plt.xticks(rotation=90)
plt.xlabel("Senador(a)")
plt.ylabel("Valor Reembolsado (Últimos 4 Anos)")
plt.title("Top 10 Senadores - Soma dos Últimos 4 Anos (Valores Abreviados)")

plt.tight_layout()
plt.show()


In [ ]:
# Filtrar os últimos 4 anos
ultimos_4_anos = sorted(dataset['ANO'].unique())[-4:]

# Somar reembolsos dos últimos 4 anos por senador
df_4_anos = dataset[dataset['ANO'].isin(ultimos_4_anos)].groupby('SENADOR')['VALOR_REEMBOLSADO'].sum().reset_index()

# Encontrar os top 10 senadores
top_10_senadores = df_4_anos.nlargest(10, 'VALOR_REEMBOLSADO')

# Calcular o total de reembolso
total_reembolso = dataset['VALOR_REEMBOLSADO'].sum()

# Calcular o reembolso dos top 10 senadores como uma porção do total
top_10_total = top_10_senadores['VALOR_REEMBOLSADO'].sum()
outros = total_reembolso - top_10_total

# Criar gráfico de pizza
plt.figure(figsize=(8, 8))
somas = [top_10_total, outros]
labels = ['Top 10 Senadores', 'Outros Senadores']
colors = ['orange', 'skyblue']
explode = (0.1, 0)  # 'explode' a slice if you want to highlight it

plt.pie(somas, labels=[f"{lab}: {abreviar_valor(val)}" for lab, val in zip(labels, somas)], colors=colors, autopct='%1.1f%%', startangle=140, explode=explode)
plt.title('Percentual dos Top 10 Senadores nos Reembolsos Totais')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()


Neste gráfico vemos que apenas 10 senadores gastaram praticamente 1/5 do valor total reembolsado para os 159 senadores.

In [ ]:
# Criar dataset sem outliers
dataset_sem_outliers = dataset.drop(outliers.index)

# Verificar o tamanho do dataset antes e depois
print(f"Tamanho original do dataset: {len(dataset)}")
print(f"Tamanho do dataset sem outliers: {len(dataset_sem_outliers)}")


In [ ]:
# Garantir que o dataset tem a coluna 'ANO' e 'VALOR_REEMBOLSADO'
# Criar a série temporal agrupando os dados por ANO
# dataset['ANO'] = pd.to_datetime(dataset['DATA'], errors='coerce').dt.year
dados_serie = dataset.groupby('ANO')['VALOR_REEMBOLSADO'].sum()

# Garantir que o índice da série está no formato datetime (opcional)
dados_serie.index = pd.to_datetime(dados_serie.index, format='%Y')

# Exibir a série para verificar se está correta
print(dados_serie)


In [ ]:
# Definir os anos considerados outliers
outliers_anos = [2020]

# Filtrar os dados para excluir os anos atípicos
dados_sem_outliers = dados_serie[~dados_serie.index.year.isin(outliers_anos)]

# Verificar os dados filtrados
print(dados_sem_outliers)


In [ ]:
# Ajustar o modelo ARIMA sem os anos atípicos
modelo_sem_outliers = ARIMA(dados_sem_outliers, order=(1, 1, 1))
modelo_ajustado_sem_outliers = modelo_sem_outliers.fit()

# Fazer previsões
previsoes_sem_outliers = modelo_ajustado_sem_outliers.forecast(steps=3)

# Visualizar as previsões
print(f"Previsões sem os anos atípicos:\n{previsoes_sem_outliers}")


In [ ]:
# Agrupar os valores reembolsados por ano
soma_ano = dataset.groupby('ANO')['VALOR_REEMBOLSADO'].sum().reset_index()

# Garantir que todos os anos estão no dataset, incluindo 2020
dados_reais = soma_ano.set_index('ANO')['VALOR_REEMBOLSADO']

# Previsões geradas pelo modelo (ajuste para 3 passos: 2023, 2024 e 2025)
forecast_result = modelo_ajustado_sem_outliers.get_forecast(steps=3)

# Obter as previsões e os intervalos de confiança
previsoes = forecast_result.predicted_mean
intervalo_confianca = forecast_result.conf_int()

# Criar índice para os anos das previsões
anos_futuros = [2023, 2024, 2025]

# Criar um DataFrame para as previsões
previsoes_df = pd.DataFrame({
    'ANO': anos_futuros,
    'VALOR_PREVISTO': previsoes.values
}).set_index('ANO')

# Adicionar os intervalos de confiança ao DataFrame
previsoes_df['IC_INFERIOR'] = intervalo_confianca.iloc[:, 0].values
previsoes_df['IC_SUPERIOR'] = intervalo_confianca.iloc[:, 1].values

# Converter valores para milhões
dados_reais_milhoes = dados_reais / 1_000_000
previsoes_df_milhoes = previsoes_df / 1_000_000
soma_ano['VALOR_REEMBOLSADO'] = soma_ano['VALOR_REEMBOLSADO'] / 1_000_000  # Convertendo para milhões

# Plotar os dados reais e as previsões com margens de erro
plt.figure(figsize=(10, 6))

# Linha dos dados reais
plt.plot(dados_reais_milhoes.index, dados_reais_milhoes.values, label='Dados Reais', marker='o')

# Linha das previsões
plt.plot(previsoes_df_milhoes.index, previsoes_df_milhoes['VALOR_PREVISTO'], label='Previsões', linestyle='--', marker='x', color='red')

# Preencher área das margens de erro
plt.fill_between(previsoes_df_milhoes.index,
                 previsoes_df_milhoes['IC_INFERIOR'],
                 previsoes_df_milhoes['IC_SUPERIOR'],
                 color='red', alpha=0.2, label='Intervalo de Confiança')

# Adicionar título e rótulos
plt.title('Previsões Incluindo o Ano 2020 (ARIMA) com Margens de Erro', fontsize=14)
plt.xlabel('Ano', fontsize=12)
plt.ylabel('Valor Reembolsado (em Milhões de R$)', fontsize=12)

# Adicionar legenda
plt.legend()

# Mostrar o gráfico
plt.grid(True)
plt.show()

# Exibir os valores reais, previsões e soma_ano
print("\nValores Reais (em Milhões de R$):")
print(dados_reais_milhoes)

print("\nPrevisões (em Milhões de R$):")
print(previsoes_df_milhoes)
